# Energy Evaluation

### Define Energy Intesities

In [4]:
# energy intensity kWh/GB
ENERGY_INTENSITY_NETWORK_KWH_PER_GB = 0.0037
ENERGY_INTENSITY_STORAGE_KWH_PER_GB = 0.0046

## Load pre-processed data
This data can be pre-processed with the help of kepler-evaluation.ipynb (CPU+RAM), network-evaluation.ipynb and storage-evaluation.ipynb. If you want to do new experiments, you have to create these files yourself in the meantioned jupyter notebooks.

In [30]:
import pandas as pd
import tabulate as tabulate
network_data = pd.read_csv("cadvisor_network_bytes_received_all_absolute_bytes.csv", index_col=0)
storage_data = pd.read_csv("cadvisor_storage_usage_all_absolute_bytes.csv", index_col=0)
kepler_data = pd.read_csv("pods_kepler_joules_all_kWh.csv", index_col=0)
kepler_data_joules = pd.read_csv("pods_kepler_joules_all_absolute_joules.csv", index_col=0)

print(network_data)

                                    label  time_normalized_rounded  \
report                                                               
report_2025-06-29_03-46-28.yaml  Baseline                     -0.0   
report_2025-06-29_03-46-28.yaml  Baseline                     30.0   
report_2025-06-29_03-46-28.yaml  Baseline                     60.0   
report_2025-06-29_03-46-28.yaml  Baseline                     90.0   
report_2025-06-29_03-46-28.yaml  Baseline                    120.0   
...                                   ...                      ...   
report_2025-06-29_08-30-47.yaml        0%                   1080.0   
report_2025-06-29_08-30-47.yaml        0%                   1110.0   
report_2025-06-29_08-30-47.yaml        0%                   1140.0   
report_2025-06-29_08-30-47.yaml        0%                   1170.0   
report_2025-06-29_08-30-47.yaml        0%                   1200.0   

                                 accounting_system-under-evaluation  \
report            

Further process the network data to transform total bytes received to kWh using the energy intensity

In [54]:
# get the last value for each label based on the time_normalized_rounded as this is the total amount of data received
network_data_processed = network_data.groupby("label").last().reset_index()

# this values are in bytes, convert them to GB
network_data_processed = network_data_processed.set_index("label") / (1024 * 1024 * 1024)

# calculate the energy consumption in kWh
network_data_processed = network_data_processed * ENERGY_INTENSITY_NETWORK_KWH_PER_GB

# remove the column time_normalized_rounded as it is not needed anymore
network_data_processed = network_data_processed.drop(columns=["time_normalized_rounded"])

# remove _system-under-evaluation from the column names
network_data_processed.columns = network_data_processed.columns.str.replace("_system-under-evaluation", "", regex=False)

network_data_processed = network_data_processed.reindex([
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "Base Persistence",
    "5% Persistence",
    "10% Persistence",
    "50% Persistence"
])

# print as table
print("Total network data received per Experiment (label) and energy consumption in kWh:")
print(network_data_processed.to_markdown())
print("Total energy consumption for storage in kWh per Experiment (label) and percentage difference from label 'Baseline':")

for label in network_data_processed.index:
    if label == "Baseline":
        print(f"{label}: {network_data_processed.loc[label].sum()} kWh (100%)")
    else:
        percentage_difference = (network_data_processed.loc[label].sum() / network_data_processed.loc["Baseline"].sum()) * 100
        print(f"{label}: {network_data_processed.loc[label].sum()} kWh ({percentage_difference:.2f}%)")

Total network data received per Experiment (label) and energy consumption in kWh:
| label            |   accounting |          ad |        cart |    checkout |    currency |   elasticsearch |   email |       flagd |   fraud-detection |   frontend |   frontend-proxy |     grafana |   image-provider |      jaeger |       kafka |   opensearch |   opentelemetry-collector |     payment |   product-catalog |   prometheus |   quote |   recommendation |   shipping |   valkey-cart |
|:-----------------|-------------:|------------:|------------:|------------:|------------:|----------------:|--------:|------------:|------------------:|-----------:|-----------------:|------------:|-----------------:|------------:|------------:|-------------:|--------------------------:|------------:|------------------:|-------------:|--------:|-----------------:|-----------:|--------------:|
| Baseline         |  3.71039e-06 | 9.57595e-05 | 0.000322079 | 2.57746e-07 | 5.34045e-08 |    nan          |       0 | 0.00

Further process the storage data to transform total bytes used to kWh using the energy intensity

In [52]:
storage_data_processed = storage_data.groupby("label").last().reset_index()

# this values are in bytes, convert them to GB
storage_data_processed = storage_data_processed.set_index("label") / (1024 * 1024 * 1024)

# calculate the energy consumption in kWh
storage_data_processed = storage_data_processed * ENERGY_INTENSITY_STORAGE_KWH_PER_GB

# remove the column time_normalized_rounded as it is not needed anymore
storage_data_processed = storage_data_processed.drop(columns=["time_normalized_rounded"])

# remove _system-under-evaluation from the column names
storage_data_processed.columns = storage_data_processed.columns.str.replace("_system-under-evaluation", "", regex=False)

# sort the index for readability in this order: Baseline, 5%, 10%, 50%, Baseline Persistence, 5% Persistence, 10% Persistence, 50% Persistence
storage_data_processed = storage_data_processed.reindex([
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "Base Persistence",
    "5% Persistence",
    "10% Persistence",
    "50% Persistence"
])

# print as table
print("Total energy consumption for storage in kWh per Experiment (label):")
print(storage_data_processed.to_markdown())
print("Total energy consumption for storage in kWh per Experiment (label) and percentage difference from label 'Baseline':")

for label in storage_data_processed.index:
    if label == "Baseline":
        print(f"{label}: {storage_data_processed.loc[label].sum()} kWh (100%)")
    else:
        percentage_difference = (storage_data_processed.loc[label].sum() / storage_data_processed.loc["Baseline"].sum()) * 100
        print(f"{label}: {storage_data_processed.loc[label].sum()} kWh ({percentage_difference:.2f}%)")


Total energy consumption for storage in kWh per Experiment (label):
| label            |   accounting |          ad |   elasticsearch |   email |   flagd |   fraud-detection |     frontend |   frontend-proxy |     grafana |   image-provider |       kafka |   opensearch |   payment |   prometheus |   recommendation |   shipping |   valkey-cart |
|:-----------------|-------------:|------------:|----------------:|--------:|--------:|------------------:|-------------:|-----------------:|------------:|-----------------:|------------:|-------------:|----------:|-------------:|-----------------:|-----------:|--------------:|
| Baseline         |            0 | 0.000186198 |      nan        |       0 |       0 |       0.000177722 |  0           |                0 | 8.14209e-06 |                0 | 0.000855218 |     0.156826 |         0 |   0.00774902 |                0 |        nan |   2.73743e-06 |
| 0%               |            0 | 0.000190672 |      nan        |       0 |       0 |       0

Further process the CPU energy data. We use the average over the experiment as the kWh

In [ ]:
kepler_data_processed = kepler_data.groupby("label").mean()

# remove the column time_normalized_rounded as it is not needed anymore
kepler_data_processed = kepler_data_processed.drop(columns=["time_normalized_rounded"])

# remove _system-under-evaluation from the column names
kepler_data_processed.columns = kepler_data_processed.columns.str.replace("_system-under-evaluation", "", regex=False)

# sort the index for readability in this order: Baseline, 5%, 10%, 50%, Baseline Persistence, 5% Persistence, 10% Persistence, 50% Persistence
kepler_data_processed = kepler_data_processed.reindex([
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "Base Persistence",
    "5% Persistence",
    "10% Persistence",
    "50% Persistence"
])

# print as table
print("Total energy consumption in kWh per Experiment (label):")
print(kepler_data_processed.to_markdown())
print("Total energy consumption in kWh per Experiment (label) and percentage difference from label 'Baseline':")

for label in kepler_data_processed.index:
    if label == "Baseline":
        print(f"{label}: {kepler_data_processed.loc[label].sum()} kWh (100%)")
    else:
        percentage_difference = (kepler_data_processed.loc[label].sum() / kepler_data_processed.loc["Baseline"].sum()) * 100
        print(f"{label}: {kepler_data_processed.loc[label].sum()} kWh ({percentage_difference:.2f}%)")

| label            |   accounting |       ad |    cart |    checkout |   configfile |   copy-default-plugins |    currency |   elasticsearch |   elasticsearch-checker |   email |    flagd |   flagd-ui |   fraud-detection |   frontend |   frontend-proxy |   grafana |   image-provider |   init-config |      jaeger |   jaeger-agent-sidecar |   jaeger-collector |   jaeger-query |    kafka |    master |   opensearch |   opentelemetry-collector |    payment |   product-catalog |   prometheus-server |   quote |   recommendation |   shipping |   sysctl |   valkey-cart |   wait-for-kafka |   wait-for-valkey-cart |   worker |
|:-----------------|-------------:|---------:|--------:|------------:|-------------:|-----------------------:|------------:|----------------:|------------------------:|--------:|---------:|-----------:|------------------:|-----------:|-----------------:|----------:|-----------------:|--------------:|------------:|-----------------------:|-------------------:|---------------

In [ ]:
kepler_data_joules_processed = kepler_data_joules.groupby("label").last()

# remove the column time_normalized_rounded as it is not needed anymore
kepler_data_joules_processed = kepler_data_joules_processed.drop(columns=["time_normalized_rounded"])

# remove _system-under-evaluation from the column names
kepler_data_joules_processed.columns = kepler_data_joules_processed.columns.str.replace("_system-under-evaluation", "", regex=False)

# sort the index for readability in this order: Baseline, 5%, 10%, 50%, Baseline Persistence, 5% Persistence, 10% Persistence, 50% Persistence
kepler_data_joules_processed = kepler_data_joules_processed.reindex([
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "Base Persistence",
    "5% Persistence",
    "10% Persistence",
    "50% Persistence"
])  

# print as table
print("Total energy consumption in Joules per Experiment (label):")
print(kepler_data_joules_processed.to_markdown())

# convert Joules to kWh
kepler_data_joules_processed = kepler_data_joules_processed / 3600000  # 1 kWh = 3.6 million Joules
print("Total energy consumption in kWh per Experiment (label) and percentage difference from label 'Baseline':")

for label in kepler_data_joules_processed.index:
    if label == "Baseline":
        print(f"{label}: {kepler_data_joules_processed.loc[label].sum()} kWh (100%)")
    else:
        percentage_difference = (kepler_data_joules_processed.loc[label].sum() / kepler_data_joules_processed.loc["Baseline"].sum()) * 100
        print(f"{label}: {kepler_data_joules_processed.loc[label].sum()} kWh ({percentage_difference:.2f}%)")


| label            |   accounting |      ad |    cart |   checkout |   configfile |   copy-default-plugins |   currency |   elasticsearch |   elasticsearch-checker |   email |   flagd |   flagd-ui |   fraud-detection |   frontend |   frontend-proxy |   grafana |   image-provider |   init-config |   jaeger |   jaeger-agent-sidecar |   jaeger-collector |   jaeger-query |   kafka |   master |   opensearch |   opentelemetry-collector |   payment |   product-catalog |   prometheus-server |   quote |   recommendation |   shipping |   sysctl |   valkey-cart |   wait-for-kafka |   wait-for-valkey-cart |   worker |
|:-----------------|-------------:|--------:|--------:|-----------:|-------------:|-----------------------:|-----------:|----------------:|------------------------:|--------:|--------:|-----------:|------------------:|-----------:|-----------------:|----------:|-----------------:|--------------:|---------:|-----------------------:|-------------------:|---------------:|--------:|-----

Categorize the energy consumption into "App", "Observability" and "System"

In [55]:
# | label            |   accounting |      ad |    cart |   checkout |   configfile |   copy-default-plugins |   currency |   elasticsearch |   elasticsearch-checker |   email |   flagd |   flagd-ui |   fraud-detection |   frontend |   frontend-proxy |   grafana |   image-provider |   init-config |   jaeger |   jaeger-agent-sidecar |   jaeger-collector |   jaeger-query |   kafka |   master |   opensearch |   opentelemetry-collector |   payment |   product-catalog |   prometheus-server |   quote |   recommendation |   shipping |   sysctl |   valkey-cart |   wait-for-kafka |   wait-for-valkey-cart |   worker |

kepler_mapping = {
    "accounting": "Application",
    "ad": "Application",
    "cart": "Application",
    "checkout": "Application",
    "configfile": "Application",
    "copy-default-plugins": "Application",
    "currency": "Application",
    "elasticsearch": "Observability",
    "elasticsearch-checker": "Observability",
    "email": "Application",
    "flagd": "Observability",
    "flagd-ui": "Observability",
    "fraud-detection": "Application",
    "frontend": "Application",
    "frontend-proxy": "Application",
    "grafana": "Observability",
    "image-provider": "Application",
    "init-config": "Application",
    "jaeger": "Observability",
    "jaeger-agent-sidecar": "Observability",
    "jaeger-collector": "Observability",
    "jaeger-query": "Observability",
    "kafka": "Application",
    "master": "Application",
    "opensearch": "Observability",
    "opentelemetry-collector": "Observability",
    "payment": "Application",
    "product-catalog": "Application",
    "prometheus-server": "Observability",
    "quote": "Application",
    "recommendation": "Application",
    "shipping": "Application",
    "sysctl": "System",
    "valkey-cart": "Application",
    "wait-for-kafka": "Application",
    "wait-for-valkey-cart": "Application",
    "worker": "Application"    
}

#| label            |   accounting |          ad |   elasticsearch |   email |   flagd |   fraud-detection |     frontend |   frontend-proxy |     grafana |   image-provider |       kafka |   opensearch |   payment |   prometheus |   recommendation |   shipping |   valkey-cart |

storage_data_mapping = {
    "accounting": "Application",
    "ad": "Application",
    "elasticsearch": "Observability",
    "email": "Application",
    "flagd": "Observability",
    "fraud-detection": "Application",
    "frontend": "Application",
    "frontend-proxy": "Application",
    "grafana": "Observability",
    "image-provider": "Application",
    "kafka": "Application",
    "opensearch": "Observability",
    "payment": "Application",
    "prometheus": "Observability",
    "recommendation": "Application",
    "shipping": "Application",
    "valkey-cart": "Application"
}

# | label            |   accounting |          ad |        cart |    checkout |    currency |   elasticsearch |   email |       flagd |   fraud-detection |   frontend |   frontend-proxy |     grafana |   image-provider |      jaeger |       kafka |   opensearch |   opentelemetry-collector |     payment |   product-catalog |   prometheus |   quote |   recommendation |   shipping |   valkey-cart |

network_data_mapping = {
    "accounting": "Application",
    "ad": "Application",
    "cart": "Application",
    "checkout": "Application",
    "currency": "Application",
    "elasticsearch": "Observability",
    "email": "Application",
    "flagd": "Observability",
    "fraud-detection": "Application",
    "frontend": "Application",
    "frontend-proxy": "Application",
    "grafana": "Observability",
    "image-provider": "Application",
    "jaeger": "Observability",
    "kafka": "Application",
    "opensearch": "Observability",
    "opentelemetry-collector": "Observability",
    "payment": "Application",
    "product-catalog": "Application",
    "prometheus": "Observability",
    "quote": "Application",
    "recommendation": "Application",
    "shipping": "Application",
    "valkey-cart": "Application"
}

"""
| label            |   accounting |      ad |    cart |   checkout |   configfile |   copy-default-plugins |   currency |   elasticsearch |   elasticsearch-checker |   email |   flagd |   flagd-ui |   fraud-detection |   frontend |   frontend-proxy |   grafana |   image-provider |   init-config |   jaeger |   jaeger-agent-sidecar |   jaeger-collector |   jaeger-query |   kafka |   master |   opensearch |   opentelemetry-collector |   payment |   product-catalog |   prometheus-server |   quote |   recommendation |   shipping |   sysctl |   valkey-cart |   wait-for-kafka |   wait-for-valkey-cart |   worker |
|:-----------------|-------------:|--------:|--------:|-----------:|-------------:|-----------------------:|-----------:|----------------:|------------------------:|--------:|--------:|-----------:|------------------:|-----------:|-----------------:|----------:|-----------------:|--------------:|---------:|-----------------------:|-------------------:|---------------:|--------:|---------:|-------------:|--------------------------:|----------:|------------------:|--------------------:|--------:|-----------------:|-----------:|---------:|--------------:|-----------------:|-----------------------:|---------:|
| Baseline         |       22.056 | 726.141 | 2421.14 |      0.627 |            0 |                    nan |       0    |          nan    |                     nan |       0 | 440.286 |          0 |           153.54  |    55544.7 |          8664.98 |   235.143 |                0 |             0 |  160.701 |                nan     |            nan     |        nan     | 470.682 |   41.418 |      7796.56 |                   6076.65 |     4.104 |           3723.98 |             1463.33 |       0 |          13894.6 |          0 |      nan |       150.936 |                0 |                      0 |  21229.7 |
| 0%               |       21.462 | 741.999 | 2464.92 |      0.459 |            0 |                    nan |       0.03 |          nan    |                     nan |       0 | 449.298 |          0 |           157.992 |    55590.7 |          8723.32 |   236.679 |                0 |             0 |   20.613 |                nan     |            nan     |        nan     | 468.504 |   42.087 |      7830.49 |                   5949.31 |     4.407 |           3778.43 |             1474.48 |       0 |          14093.6 |          0 |      nan |       152.991 |                0 |                      0 |  21378.4 |
| 5%               |       20.634 | 806.406 | 2630.3  |      0.639 |            0 |                    nan |       0    |          nan    |                     nan |       0 | 455.493 |          0 |           157.989 |    55565.4 |          8901.28 |   240.357 |                0 |             0 |  302.976 |                nan     |            nan     |        nan     | 495.114 |   42.921 |      7525.52 |                   6296.35 |     4.473 |           3849.41 |             1470.78 |       0 |          14294.5 |          0 |      nan |       163.599 |                0 |                      0 |  21622.7 |
| 10%              |       20.895 | 734.727 | 2434.7  |      0.546 |            0 |                    nan |       0    |          nan    |                     nan |       0 | 447.231 |          0 |           155.25  |    55355.7 |          8720.33 |   237.639 |                0 |             0 |  447.192 |                nan     |            nan     |        nan     | 484.374 |   40.842 |      7756.51 |                   6164.81 |     4.281 |           3784.8  |             1472.51 |       0 |          13982.9 |          0 |      nan |       150.621 |                0 |                      0 |  21413.1 |

"""

def categorize_energy_consumption(data: pd.DataFrame, mapping: dict) -> pd.DataFrame:
    """
    Categorize and sum energy consumption based on component categories.

    Parameters:
    - data (pd.DataFrame): A DataFrame where rows represent measurements and columns represent component labels.
    - mapping (dict): A mapping from component label (column name) to a category (e.g., "Application", "Observability").

    Returns:
    - pd.DataFrame: A DataFrame with the same row indices as `data` and columns being the summed energy per category.
    """
    # Initialize a DataFrame to store summed results
    categorized = pd.DataFrame(index=data.index)

    # Iterate through unique categories
    for category in set(mapping.values()):
        # Select columns belonging to this category
        cols = [col for col in data.columns if mapping.get(col) == category]
        # Sum across these columns and store in the result
        if cols:
            categorized[category] = data[cols].sum(axis=1)
        else:
            categorized[category] = 0  # If no columns match the category, fill with 0

    return categorized

# Categorize the energy consumption for each dataset
network_categorized = categorize_energy_consumption(network_data_processed, network_data_mapping)
storage_categorized = categorize_energy_consumption(storage_data_processed, storage_data_mapping)
kepler_categorized = categorize_energy_consumption(kepler_data_processed, kepler_mapping)

# Print the categorized energy consumption
print("\nNetwork Energy Consumption Categorized:")
print(network_categorized.to_markdown())
print("\nStorage Energy Consumption Categorized:")
print(storage_categorized.to_markdown())
print("\nKepler Energy Consumption Categorized:")
print(kepler_categorized.to_markdown())

# merge the categorized dataframes and sum on same column name
categorized_combined = pd.concat([network_categorized, storage_categorized, kepler_categorized], axis=1)
categorized_combined = categorized_combined.groupby(categorized_combined.columns, axis=1).sum()


# Print the combined categorized energy consumption
print("\nCombined Categorized Energy Consumption:")
print(categorized_combined.to_markdown())

# add total summing up the categories for each experiment (label)
categorized_combined["Total"] = categorized_combined.sum(axis=1)

categorized_combined_transpose = categorized_combined.transpose()
print(categorized_combined_transpose.to_markdown())




Network Energy Consumption Categorized:
| label            |   Application |   Observability |
|:-----------------|--------------:|----------------:|
| Baseline         |     0.0157113 |       0.0347136 |
| 0%               |     0.0158559 |       0.0346125 |
| 5%               |     0.016017  |       0.0352575 |
| 10%              |     0.0158597 |       0.0342499 |
| 50%              |     0.015883  |       0.0357424 |
| 1s               |     0.0157119 |       0.0353469 |
| Base Persistence |     0.0154478 |       0.0366071 |
| 5% Persistence   |     0.0150311 |       0.0418064 |
| 10% Persistence  |     0.0149926 |       0.0476466 |
| 50% Persistence  |     0.0143195 |       0.0753032 |

Storage Energy Consumption Categorized:
| label            |   Application |   Observability |
|:-----------------|--------------:|----------------:|
| Baseline         |    0.00122188 |        0.164583 |
| 0%               |    0.00121912 |        0.165876 |
| 5%               |    0.00116425 |  

/tmp/ipykernel_3481804/2767750641.py:145: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  categorized_combined = categorized_combined.groupby(categorized_combined.columns, axis=1).sum()
